In [133]:
# Initial imports
import os
import pandas as pd 
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [134]:
load_dotenv()

True

In [135]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = os.getenv("BASE_URL")

In [136]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url,
    api_version="v2")

In [137]:
# Set the ticker
symbol = "AAPL"

# Set timeframe to "1Day"
timeframe = "1Day"

# Set start and end datetimes of 4 years from Today
start_date = pd.Timestamp("2016-07-05", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-07-05", tz="America/New_York").isoformat()

# Get 1 year's worth of historical data for AAPL
historical_data = api.get_bars(
    symbol,
    timeframe,
    start=start_date,
    end=end_date
).df

In [138]:
#Define Trading Strategy
#Retrieve historical data for a specific stock symbol
def trading_strategy():
    symbol = 'AAPL'
    timeframe = '1D' 
    limit = 100
    historical_data = api.get_bars(
    symbol,
    timeframe,
    start=start_date,
    end=end_date
).df

In [139]:
#Calculate SMA indicators
historical_data['SMA30'] = historical_data['close'].rolling(window=30).mean()
historical_data['SMA50'] = historical_data['close'].rolling(window=50).mean() 

In [140]:
#Determine latest SMA values
latest_sma30 = historical_data['SMA30'].iloc[-1]
latest_sma50 = historical_data['SMA50'].iloc[-1]

In [141]:
#Check for trading signals
#Example: Buy 10 shares of AAPL if SMA30 crosses above SMA 50 
if latest_sma30 > latest_sma50:
    qty = 10 
    side = 'buy'
    order_type = 'market'
    api.submit_order(
    symbol=symbol,
    qty=qty,
    side=side,
    type=order_type,
    time_in_force='gtc'
)
elif latest_sma30 < latest50:
    qty = 'all'
    side = 'sell'
    order_type ='market'
    api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type=order_type,
        time_in_force='gtc'
)

In [142]:
#Run the trading bot
if __name__ == '__main__':
    trading_strategy()